## Gemini PDF Processing and Validation Script  
#### Automated extraction of mining resource tables using Google Gemini API

---

#### Overview
This script uses **Google’s Gemini API** to extract **structured mineral resource data** from filtered PDF pages (e.g., those produced by the Unstructured processing stage).  
It prompts Gemini to output clean, pipe-delimited (`|`) text files that adhere to a predefined schema for **mineral resource estimation (MRE)** data.

---

#### Core Functions
##### `validate_output_file(file_path)`
Validates the generated `.txt` output for data integrity and schema compliance:
- Ensures correct **column count and order**
- Checks valid **deposit_type** and **mineral_resource** values
- Confirms that numeric fields (`tonnage_kt`, `grade_gt`) contain valid numbers

##### `process_pdfs_with_gemini(input_folder, output_folder, max_retries, retry_delay)`
Coordinates the main Gemini-based extraction process:
1. Initializes the Gemini client using your `GEMINI_API_KEY`  
2. Iterates through all PDFs in the input folder  
3. Sends each file (plus an extraction prompt) to Gemini for structured output  
4. Saves the `.txt` output and runs validation  
5. Retries failed extractions (up to `max_retries`) before moving invalid results to a `failed_validation` folder  

---

#### 📂 Expected Input and Output
**Input folder:**  
`unstructured/pdf_filtered/` — filtered PDFs from the Unstructured pipeline  

**Output folder:**  
`unstructured/gemini/normalized_output/` — validated `.txt` outputs  

**Failed validation folder:**  
`unstructured/gemini/normalized_output/failed_validation/`  

---

#### Output Format
Each line in the output `.txt` file follows this **pipe-delimited schema**:

| Column | Description |
|:--|:--|
| `project_name` | Name of the mining project |
| `deposit_name` | Deposit or orebody name |
| `deposit_type` | One of: open-pit, underground, heap, stockpile, other, unknown |
| `date_updated` | Report date (ISO format preferred) |
| `mineral_resource` | One of: inferred, indicated, measured, total |
| `cutoff_gt` | Cutoff grade (grams per tonne) |
| `mineral` | Primary mineral (symbol) |
| `mineral_list` | List of minerals (e.g., Au-Cu-Zn) |
| `tonnage_kt` | Ore tonnage (kilotonnes) |
| `grade_gt` | Average grade (grams per tonne) |

---

#### Workflow Summary
1. Define validation schema (`EXPECTED_COLUMNS`) and valid categories  
2. Send filtered PDFs to Gemini for extraction  
3. Validate each generated output  
4. Retry or move invalid outputs to `failed_validation`  
5. Produce clean, structured text ready for data ingestion  

---

#### 🔑 Environment Variable
Ensure your **Gemini API key** is set before running:
```bash
export GEMINI_API_KEY="your_api_key_here"


In [ ]:
from google import genai
import pathlib
import os
import time
import shutil

EXPECTED_COLUMNS = [
    'project_name', 'deposit_name', 'deposit_type', 'date_updated',
    'mineral_resource', 'cutoff_gt', 'mineral', 'mineral_list',
    'tonnage_kt', 'grade_gt'
]

VALID_DEPOSIT_TYPES = ['open-pit', 'underground', 'heap', 'stockpile', 'other', 'unknown']
VALID_RESOURCE_TYPES = ['inferred', 'indicated', 'measured', 'total']

def validate_output_file(file_path):
    """
    Validate the TXT output for data quality.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    for i, line in enumerate(lines):
        parts = line.split('|')

        # Check column count
        if len(parts) != len(EXPECTED_COLUMNS):
            return False, f"Invalid column count on line {i+1}: {len(parts)}"

        # Skip header line if it matches expected columns
        if i == 0 and set(parts) == set(EXPECTED_COLUMNS):
            continue

        project_name, deposit_name, deposit_type, date_updated, mineral_resource, cutoff_gt, mineral, mineral_list, tonnage_kt, grade_gt = parts

        # Check deposit_type validity
        if deposit_type.lower() not in VALID_DEPOSIT_TYPES and deposit_type != '':
            return False, f"Invalid deposit_type on line {i+1}: {deposit_type}"

        # Check mineral_resource validity
        if mineral_resource.lower() not in VALID_RESOURCE_TYPES and mineral_resource != '':
            return False, f"Invalid mineral_resource on line {i+1}: {mineral_resource}"

        # Check numeric values
        if tonnage_kt and not is_numeric(tonnage_kt):
            return False, f"Non-numeric tonnage_kt on line {i+1}: {tonnage_kt}"

        if grade_gt and not is_numeric(grade_gt):
            return False, f"Non-numeric grade_gt on line {i+1}: {grade_gt}"

    return True, "Validation passed"

def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def process_pdfs_with_gemini(input_folder, output_folder, max_retries, retry_delay):
    try:
        # Initialize Gemini client
        client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))

        # Create output and failed directories if they don't exist
        os.makedirs(output_folder, exist_ok=True)
        failed_folder = os.path.join(output_folder, 'failed_validation')
        os.makedirs(failed_folder, exist_ok=True)

        # Define the prompt
        prompt = (
            "You are a data extraction agent specialized in mining reports."
            "The following is a subset of pages containing tables from a mining report PDF."
            "From the document, output txt (use | delimiter) with the following columns: "
            "'project_name', 'deposit_name', 'deposit_type', 'date_updated', 'mineral_resource', 'cutoff_gt', 'mineral', 'mineral_list',"
            "'tonnage_kt', and 'grade_gt'."
            "Only extract data related to mineral resources, skip from ore reserves table (skip prove/probable)."
            "deposit_type is 'open-pit', 'underground', 'heap', 'stockpile', 'other', 'unknown'."
            "mineral_resource is 'inferred', 'indicated', 'measured', or 'total'."
            "cutoff is primarily for gold, unknown if not specified."
            "mineral is atomic symbol like ag,au,bi,co,cu,fe,in,mo,pb,pd,pt,re,s,sb,sg,ta2o5,wo3,zn."
            "mineral list is list of mineral symbol separated by dash -."
            "Convert tonnage to kt kiloton if needed. Convert grade to gt grams per tonne if needed."
            "If multiple date_updated exists for a project, deposit, and deposit_type, use only data from the latest date."
            "Stick to the predefined column values for deposit_type and mineral_resource, do not use any other variations."
            "Do not include subtotals or totals from the tables, unless they are the only values present."
            "Output empty values if none found. Stick to the column structure, do not include any other text or explanation, just the CSV output."
        )

        for filename in os.listdir(input_folder):
            if filename.lower().endswith('.pdf'):
                file_path = pathlib.Path(os.path.join(input_folder, filename))
                output_filename = os.path.splitext(filename)[0] + '.txt'
                output_path = os.path.join(output_folder, output_filename)

                if os.path.exists(output_path):
                    print(f"Skipping {filename} (output already exists: {output_path})")
                    continue

                print(f"Processing file: {file_path}")
                retries = 0

                while retries < max_retries:
                    try:
                        # Upload the PDF using the File API
                        sample_file = client.files.upload(file=file_path)

                        # Generate content from the model
                        response = client.models.generate_content(
                            model="gemini-2.5-flash",
                            contents=[sample_file, prompt]
                        )

                        # Save the response to a text file
                        with open(output_path, 'w', encoding='utf-8') as f:
                            f.write(response.text)
                        print(f"Saved output to: {output_path}")

                        # Validate the output
                        valid, message = validate_output_file(output_path)
                        if valid:
                            print("Validation passed ✅")
                            break
                        else:
                            print(f"Validation failed: {message}")
                            # Delete invalid file
                            os.remove(output_path)
                            retries += 1
                            if retries < max_retries:
                                print(f"Retrying extraction (Attempt {retries}/{max_retries}) in {retry_delay} seconds...")
                                time.sleep(retry_delay)
                            else:
                                print(f"Max retries reached for {filename}. Moving to failed folder.")
                                failed_path = os.path.join(failed_folder, output_filename)
                                with open(failed_path, 'w', encoding='utf-8') as f:
                                    f.write(f"FAILED VALIDATION after {max_retries} attempts.\nLast error: {message}\n")
                                print(f"Moved failed log to: {failed_path}")

                    except Exception as e:
                        print(f"Error processing {filename}: {e}")
                        if "503" in str(e):
                            retries += 1
                            print(f"Retrying in {retry_delay} seconds... (Attempt {retries}/{max_retries})")
                            time.sleep(retry_delay)
                        else:
                            print("Skipping retries for a non-transient error.")
                            break

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    input_folder = 'unstructured/pdf_filtered'
    output_folder = 'unstructured/gemini/normalized_output'
    max_retries = 3
    retry_delay = 5
    
    process_pdfs_with_gemini(input_folder, output_folder, max_retries, retry_delay)

### Data Cleaning and Consolidation Script  
### Combine and validate Gemini outputs for structured MRE dataset

---

#### Overview
This script cleans, normalizes, and merges all Gemini-extracted `.txt` files into a single Excel file.  
It ensures consistent column structure, valid date formats, and removes unnecessary total/overall rows for **gold (Au)** deposits.

---

#### Key Steps
1. **Read and validate** all `.txt` files from  
   `unstructured/gemini/normalized_output/`
2. **Normalize dates** using flexible parsing (handles `year`, `month-year`, or full dates)  
3. **Combine** all valid rows into a single DataFrame  
4. **Add metadata columns**:
   - `filename` — source file name  
   - `key` — unique composite key  
   - `number_of_mineral` — count of listed minerals  
5. **Filter Au dataset**  
   - Removes `total`, `overall`, `subtotal` entries if detailed resources exist  
6. **Save output** to:
   - `Combined` — all valid rows  
   - `Filtered_Au` — cleaned gold data only  
   - `Log` — records errors or mismatched rows  

---

#### Output
📁 **File:** `combined_output.xlsx`  
Contains three sheets:  
- **Combined** → all processed data  
- **Filtered_Au** → cleaned gold subset  
- **Log** → issues or encoding errors  

---

#### ✅ Notes
- Handles inconsistent encodings gracefully  
- Ensures date normalization with fallback for incomplete values  
- Designed for **Gemini MRE outputs** with a 10-column expected schema  

---

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime
from dateutil import parser
from calendar import monthrange
import numpy as np

# === CONFIG ===
input_folder = "unstructured/gemini/normalized_output"
output_file = "combined_output.xlsx"

# Expected header
expected_header = [
    "project_name", "deposit_name", "deposit_type", "date_updated",
    "mineral_resource", "cutoff_gt", "mineral", "mineral_list", "tonnage_kt", "grade_gt"
]
expected_col_count = len(expected_header)


# === FUNCTIONS ===
def normalize_date(date_str):
    if not date_str or date_str.strip().lower() in ["unknown", "n/a", "none"]:
        return None  # Return None for missing values
    
    # Split only on " and " or ";", NOT comma
    parts = re.split(r'\s+and\s+|;', date_str.strip(), flags=re.IGNORECASE)
    date_candidates = []
    
    for part in parts:
        part = part.strip()
        if not part:
            continue
        
        # Skip incomplete date fragments (must contain year)
        if not re.search(r'\d{4}', part):
            continue
        
        try:
            parsed_date = parser.parse(part, dayfirst=True, fuzzy=True)
            date_candidates.append(parsed_date)
        except:
            # Handle year only
            if re.fullmatch(r'\d{4}', part):
                y = int(part)
                date_candidates.append(datetime(y, 12, 31))
            elif re.match(r'^[A-Za-z]{3,9}[- ]?\d{4}$', part):  # Month Year
                try:
                    temp = parser.parse(part, fuzzy=True)
                    last_day = monthrange(temp.year, temp.month)[1]
                    date_candidates.append(datetime(temp.year, temp.month, last_day))
                except:
                    continue
    
    if not date_candidates:
        return None
    
    return max(date_candidates)  # Return datetime object


# === MAIN PROCESS ===
all_rows = []
error_log = []

for file_name in os.listdir(input_folder):
    if file_name.lower().endswith(".txt"):
        file_path = os.path.join(input_folder, file_name)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
        except UnicodeDecodeError:
            error_log.append(f"File: {file_name} - Encoding issue")
            continue
        
        for idx, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue
            
            parts = [p.strip() for p in line.split("|")]
            
            # Remove header row if present
            if idx == 0 and [p.lower() for p in parts] == [h.lower() for h in expected_header]:
                continue
            
            # Validate column count
            if len(parts) != expected_col_count:
                error_log.append(f"File: {file_name}, Row: {idx+1}, Columns: {len(parts)} (expected {expected_col_count})")
                continue
            
            # Normalize date (4th column)
            parts[3] = normalize_date(parts[3])
            
            all_rows.append(parts + [file_name])

# Add filename to header
output_header = expected_header + ["filename"]

# Create DataFrame
df = pd.DataFrame(all_rows, columns=output_header)

# Convert date_updated to datetime and allow NaT
df["date_updated"] = pd.to_datetime(df["date_updated"], errors="coerce")

# === Add extra columns ===
df["key"] = df["project_name"].str.lower().str.strip() + "|" + \
            df["deposit_name"].str.lower().str.strip() + "|" + \
            df["deposit_type"].str.lower().str.strip() + "|" + \
            df["date_updated"].astype(str) + "|" + \
            df["mineral"].str.lower().str.strip()

df["number_of_mineral"] = df["mineral_list"].apply(lambda x: len(x.split("-")) if isinstance(x, str) and x.strip() else 0)

# === Filter for Au FIRST ===
df_au = df[df["mineral"].str.lower() == "au"].copy()

# === Remove rows with 'total' or 'overall' in deposit_name for Au data only ===
# Create a grouping key for Au data (without date_updated to group related deposits)
df_au["grouping_key"] = df_au["project_name"].str.lower().str.strip() + "|" + \
                        df_au["deposit_name"].str.lower().str.strip() + "|" + \
                        df_au["deposit_type"].str.lower().str.strip()

# Identify rows with 'total' or 'overall' in deposit_name (including sub-total, subtotal, etc.)
has_total_overall = df_au["deposit_name"].str.lower().str.contains("total|overall|sub-total|subtotal", na=False)

# Group by the grouping_key to check for detailed resources
rows_to_keep = []

for grouping_key, group in df_au.groupby("grouping_key"):
    # Check if this group has any measured/indicated/inferred mineral resources
    has_detailed_resources = group["mineral_resource"].str.lower().isin(["inferred", "indicated", "measured"]).any()
    
    if has_detailed_resources:
        # If group has detailed resources anywhere, remove total/overall rows
        filtered_group = group[~has_total_overall.loc[group.index]]
        print(f"Removed total/overall from Au group: {grouping_key[:50]}...")
    else:
        # If group doesn't have any detailed resources, keep all rows (including total/overall)
        filtered_group = group
        print(f"Kept all rows for Au group: {grouping_key[:50]}...")
    
    rows_to_keep.append(filtered_group)

# Combine all filtered Au groups
df_au = pd.concat(rows_to_keep, ignore_index=True)

print(f"Au dataset has {len(df_au)} rows")
print(f"Au rows with total/overall remaining: {df_au['deposit_name'].str.lower().str.contains('total|overall', na=False).sum()}")

# === Save to Excel ===
with pd.ExcelWriter(output_file, engine="openpyxl", date_format="YYYY-MM-DD") as writer:
    df.to_excel(writer, index=False, sheet_name="Combined")
    df_au.to_excel(writer, index=False, sheet_name="Filtered_Au")
    
    log_df = pd.DataFrame(error_log if error_log else ["No errors"], columns=["Log"])
    log_df.to_excel(writer, index=False, sheet_name="Log")

print(f"✅ Combined and filtered files saved to {output_file}")
if error_log:
    print("⚠ Some rows had issues. Check 'Log' sheet in the Excel file.")
else:
    print("✅ All rows processed successfully.")

### MRE Wide-Format Transformation Script  
#### Convert long-format dataset into wide-format with grade, tonnage, and gold value

---

#### Summary
This script reshapes the prepared MRE dataset by:
- Pivoting **grade** and **tonnage** by resource category  
- Calculating **total tonnage** and **weighted average grade** when missing  
- Computing **gold values (oz)** for each resource type  
- Exporting the final wide-format dataset to Excel  

**Input:** `prepared_dataset.xlsx`  
**Output:** `prepared_dataset_wide_mre_with_goldval.xlsx`

---


In [ ]:
import pandas as pd
import numpy as np

# === CONFIG ===
input_file = "prepared_dataset.xlsx"  # Replace with your file
output_file = "prepared_dataset_wide_mre_with_goldval.xlsx"

# === READ DATA ===
df = pd.read_excel(input_file)
df.columns = df.columns.str.strip()

# === NORMALIZE mineral_resource ===
df['mineral_resource'] = df['mineral_resource'].str.strip().str.lower()

# === CREATE GRADE WIDE COLUMNS ===
grade_wide = df.pivot_table(
    index=["asx_code", "company_name", "project_name", "deposit_name", "deposit_type"],
    columns="mineral_resource",
    values="grade_gt",
    aggfunc='first'
).reset_index()
grade_wide.columns.name = None
grade_wide = grade_wide.rename(columns={
    "indicated": "grade_indicated_gt",
    "measured": "grade_measured_gt",
    "inferred": "grade_inferred_gt",
    "total": "grade_total_gt"
})

# === CREATE TONNAGE WIDE COLUMNS ===
tonnage_wide = df.pivot_table(
    index=["asx_code", "company_name", "project_name", "deposit_name", "deposit_type"],
    columns="mineral_resource",
    values="tonnage_kt",
    aggfunc='first'
).reset_index()
tonnage_wide.columns.name = None
tonnage_wide = tonnage_wide.rename(columns={
    "indicated": "tonnage_indicated_kt",
    "measured": "tonnage_measured_kt",
    "inferred": "tonnage_inferred_kt",
    "total": "tonnage_total_kt"
})

# === MERGE WIDE DATA BACK WITH OTHER COLUMNS ===
desc_cols = ["filename", "announcement_date", "date_updated", "loc_country", "loc_state",
             "loc_district", "loc_city", "loc_coordinates", "loc_notes",
             "project_status", "source_of_funding", "funding_notes", "funding_party"]

desc_df = df.groupby(["asx_code", "company_name", "project_name", "deposit_name", "deposit_type"])[desc_cols].first().reset_index()

wide_df = grade_wide.merge(tonnage_wide, on=["asx_code", "company_name", "project_name", "deposit_name", "deposit_type"])
wide_df = wide_df.merge(desc_df, on=["asx_code", "company_name", "project_name", "deposit_name", "deposit_type"])

# === FILL total_kt if missing ===
kt_cols = ["tonnage_indicated_kt", "tonnage_measured_kt", "tonnage_inferred_kt"]
wide_df["tonnage_total_kt"] = wide_df["tonnage_total_kt"].fillna(wide_df[kt_cols].sum(axis=1, skipna=True))

# === FILL total_gt if missing (weighted average) ===
def weighted_average(row):
    if pd.notna(row["grade_total_gt"]):
        return row["grade_total_gt"]
    num = 0
    denom = 0
    for gt_col, kt_col in zip(["grade_indicated_gt","grade_measured_gt","grade_inferred_gt"], 
                              ["tonnage_indicated_kt","tonnage_measured_kt","tonnage_inferred_kt"]):
        if pd.notna(row[gt_col]) and pd.notna(row[kt_col]):
            num += row[gt_col] * row[kt_col]
            denom += row[kt_col]
    if denom > 0:
        return num / denom
    else:
        return np.nan

wide_df["grade_total_gt"] = wide_df.apply(weighted_average, axis=1)

# === COMPUTE GOLD VALUE (in ounces) ===
for res in ["indicated", "measured", "inferred", "total"]:
    grade_col = f"grade_{res}_gt"
    tonnage_col = f"tonnage_{res}_kt"
    goldval_col = f"goldval_{res}_oz"
    wide_df[goldval_col] = wide_df[grade_col] * wide_df[tonnage_col] * 32.1507  # kt * g/t -> oz

# === WRITE TO EXCEL ===
wide_df.to_excel(output_file, index=False)
print(f"Wide-format MRE data with gold value columns written to {output_file}")